In [1]:
import Pkg
pkgs = [
"DataFrames",
"Dates",
"uCSV",
"Revise",
]
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [ ]:
# mamba create -n mmseqs2 -c bioconda mmseqs2

# conda run --no-capture-output -n mmseqs2 mmseqs 

In [ ]:
# MMSeqs Easy taxonomy
# amino acid
# ~ 5 hour download time may not be tolerable?
# processing
# 7930.350543 seconds (710.71 k allocations: 40.309 MiB, 0.00% gc time, 0.00% compilation time)
# 2 hours 12m
# 1.5 hours for iterative search with varying sensitivity
# Mycelia.download_mmseqs_db(db="UniRef100", conda_env="viral-pangenome-discovery")
# --force-reuse 1
# --remove-tmp-files 1
# conda run --live-stream -n viral-pangenome-discovery

# download on NERSC is way too slow - going to skip for now but will consider retrying later
# mkdir -p $HOME/workspace/mmseqs
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef100 $HOME/workspace/mmseqs/UniRef100 $HOME/workspace/mmseqs/tmp
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef90 $HOME/workspace/mmseqs/UniRef90 $HOME/workspace/mmseqs/tmp
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef50 $HOME/workspace/mmseqs/UniRef50 $HOME/workspace/mmseqs/tmp

In [2]:
data_dir = joinpath(dirname(pwd()), "data")

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data"

In [3]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)
# filter out samples that have already been processed
# SRR_paths = filter(x -> !isfile(joinpath(x, basename(x) * ".contig_info.tsv")), SRR_paths)

594-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399459"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399460"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399461"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399462"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399463"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399464"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399465"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399466"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399467"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399468"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399469"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/

In [7]:
target_database = "$(homedir())/workspace/mmseqs/UniRef100"
for SRR_path in SRR_paths[1:1]
    SRR = basename(SRR_path)
    assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")
    out_dir = mkpath(joinpath(SRR_path, "mmseqs_easy_search"))
    outfile_base = joinpath(out_dir, basename(assembled_fasta) * ".mmseqs_easy_search." * basename(target_database))
    run(`sbatch $(homedir())/workspace/Mycelia/scripts/mmseqs-easy-search.sh $assembled_fasta $target_database $out_dir`)
    # println("conda run --no-capture-output -n mmseqs2 mmseqs easy-taxonomy $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))")
    # Mycelia.run_mmseqs_easy_search(out_dir=SRR_path, query_fasta=assembled_fasta, target_database=target_database, outfile=SRR)
end

Submitted batch job 42295280
